In [5]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-

import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

from keras import backend as K
from keras.preprocessing import image
import pandas as pd
import numpy as np
from scipy.misc import imresize
import glob
import os

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [6]:
train_img=[]
test_img=[]
TRAIN_DIR='/dogsvcats/'
train_cats = sorted(glob.glob(os.path.join(TRAIN_DIR, 'cat*.jpg')))
train_dogs = sorted(glob.glob(os.path.join(TRAIN_DIR, 'dog*.jpg')))
train_all = train_cats[:10000] + train_dogs[:10000]
test_all = train_cats[10000:] + train_dogs[10000:]
train_label = []
test_label = []

In [7]:
for i in range(len((train_all))):
    path=train_all[i]
    temp_img=image.load_img(path,target_size=(128,128))
    temp_img=image.img_to_array(temp_img)
    if path.find('/cat.') != -1:
        train_label.append(0)
    else:
        train_label.append(1)
    train_img.append(temp_img)
    
for i in range(len((test_all))):
    path=test_all[i]
    temp_img=image.load_img(path,target_size=(128,128))
    temp_img=image.img_to_array(temp_img)
    if path.find('/cat.') != -1:
        test_label.append(0)
    else:
        test_label.append(1)
    test_img.append(temp_img)

train_img=np.array(train_img) 
test_img=np.array(test_img) 

train_y=np.asarray(train_label)
train_y=pd.get_dummies(train_y)
train_y=np.array(train_y)


test_y=np.asarray(test_label)
test_y=pd.get_dummies(test_y)
test_y=np.array(test_y)



In [8]:
train_img=preprocess_input(train_img)
test_img=preprocess_input(test_img)



In [40]:
def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):

    model = VGG16(weights='imagenet', include_top=False,input_shape=(128,128,3))
    model.layers.pop()


    model.outputs = [model.layers[-1].output]

    model.layers[-2].outbound_nodes = []
    x=Conv2D(256, kernel_size=(2,2),strides=2)(model.output)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)    
    x=Conv2D(128, kernel_size=(2,2),strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x=Flatten()(x)
    x=Dense(num_classes, activation='softmax')(x)

    model=Model(model.input,x)

    for layer in model.layers[:19]:

        layer.trainable = False

    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

    return model



In [41]:
batch_size=128
num_classes=2
epochs=15
input_shape=(128,128,3)
img_rows,img_cols=128,128
channel=3

from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.1, random_state=42)

model = vgg16_model(img_rows, img_cols, channel, num_classes)
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [42]:
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_valid,Y_valid))

Train on 18000 samples, validate on 2000 samples
Epoch 1/5
18000/18000 [==============================] - 131s - loss: 0.1135 - acc: 0.9541 - val_loss: 0.1009 - val_acc: 0.9665
Epoch 2/5
18000/18000 [==============================] - 131s - loss: 0.0450 - acc: 0.9847 - val_loss: 0.0993 - val_acc: 0.9635
Epoch 3/5
18000/18000 [==============================] - 131s - loss: 0.0231 - acc: 0.9920 - val_loss: 0.0996 - val_acc: 0.9675
Epoch 4/5
18000/18000 [==============================] - 131s - loss: 0.0160 - acc: 0.9944 - val_loss: 0.1201 - val_acc: 0.9630
Epoch 5/5
18000/18000 [==============================] - 131s - loss: 0.0084 - acc: 0.9978 - val_loss: 0.1186 - val_acc: 0.9660


In [43]:
score = model.evaluate(test_img, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.142094750365
Test accuracy: 0.962


In [44]:
model.save_weights("/output/vgg16_1.h5")
